In [ ]:
!pip install tensorflow

In [ ]:
#Importing Libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [ ]:
# Dataset Paths
data_folder = "/content/drive/MyDrive/Design Lab/Database/signals"
ctu_df_path = "/content/drive/MyDrive/Design Lab/Database/ctu_df.csv"

In [ ]:
# Parameters
sequence_length = 1500
feature_dim = 2

In [ ]:
# Load annotation file
ctu_df = pd.read_csv(ctu_df_path, dtype={'ID': str})  # Taking ID as a string
valid_ids = set(ctu_df['ID'].values)

In [ ]:
X_list = []
y_list = []

In [ ]:
# Processing each CSV file(552 files)
for file in sorted(os.listdir(data_folder)):
    if file.endswith(".csv"):
        file_id = file[:-4]  # Removing ".csv" to get ID
        if file_id not in valid_ids:
            continue  # Skip files not in ctu_df

        # Read CSV (only FHR, UC columns)
        df = pd.read_csv(os.path.join(data_folder, file), usecols=['FHR', 'UC'])
        df.dropna(inplace=True)

        # Ensure enough data points
        if len(df) < sequence_length:
            print(f"Skipping {file}, not enough data.")
            continue

        # Select the first `sequence_length` rows
        sequence = df.iloc[:sequence_length].values
        X_list.append(sequence)

        # Assign label from ctu_df
        y_list.append(ctu_df.loc[ctu_df['ID'] == file_id, 'target'].values[0])


In [ ]:
# Convert lists to NumPy arrays
X = np.array(X_list, dtype=np.float32)
y = np.array(y_list, dtype=np.int8)

print(f"Final input shape: {X.shape}")  # Should be (552, sequence_length, 2)
print(f"Final output shape: {y.shape}")  # Should be (552,)

# Save for training
np.save("X.npy", X)
np.save("y.npy", y)

Final input shape: (552, 1500, 2)
Final output shape: (552,)


In [ ]:
# Flatten X for SMOTE (Convert 3D -> 2D)
n_samples, sequence_length, num_features = X.shape
X_flat = X.reshape(n_samples, sequence_length * num_features)

In [ ]:
# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_flat, y)

In [ ]:
# Reshape X back to 3D (time series format)
X_resampled = X_resampled.reshape(-1, sequence_length, num_features)
print(f"Balanced dataset shape: {X_resampled.shape}, {y_resampled.shape}")

Balanced dataset shape: (1012, 1500, 2), (1012,)


In [ ]:
# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [ ]:
# LSTM Model
def build_lstm_model(sequence_length, num_features):
    model = keras.Sequential([
        layers.Input(shape=(sequence_length, num_features)),
        layers.LSTM(64, return_sequences=True),
        layers.LSTM(32),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")  # Binary classification
    ])
    return model

In [ ]:
# Compiling the model
model = build_lstm_model(sequence_length, num_features)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# Train Model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.5804 - loss: 0.6777 - val_accuracy: 0.6749 - val_loss: 0.5932
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.7212 - loss: 0.5503 - val_accuracy: 0.6798 - val_loss: 0.5660
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.7498 - loss: 0.4855 - val_accuracy: 0.7438 - val_loss: 0.4650
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - accuracy: 0.7554 - loss: 0.4746 - val_accuracy: 0.7291 - val_loss: 0.4793
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - accuracy: 0.7488 - loss: 0.4617 - val_accuracy: 0.7537 - val_loss: 0.4735
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.7592 - loss: 0.4508 - val_accuracy: 0.7586 - val_loss: 0.4718
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.8231 - loss: 0.3953 - val_accuracy: 0.7734 - val_loss: 0.4520
Epoch 8/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.8085 - loss: 0.4254 - val_accuracy: 0.7783 -

In [ ]:
# Evaluating the model (Calculating the model Loss and Accuracy)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7516 - loss: 0.4322
Test Accuracy: 0.7586


In [ ]:
# Evaluate Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary (0 or 1)

# Print classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Pathological", "Normal"]))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step

Classification Report:
               precision    recall  f1-score   support

Pathological       0.73      0.81      0.77       102
      Normal       0.79      0.70      0.74       101

    accuracy                           0.76       203
   macro avg       0.76      0.76      0.76       203
weighted avg       0.76      0.76      0.76       203

